#### import libraries

In [7]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import random
from itertools import cycle
from urllib.parse import urlencode

#### Get URL

In [12]:
def get_url(position, location):
    template = 'https://uk.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

url = get_url('software+consultant', 'london')

#### Establish proxy

In [13]:
proxy_params = {
      'api_key': '78923aa5-caac-440a-a88f-d1105626fd12',
      'url': url, 
      'render_js': True,
  }

response = requests.get(
  url='https://proxy.scrapeops.io/v1/',
  params=urlencode(proxy_params),
  timeout=120,
)

response

<Response [200]>

#### Create BeautifulSoup object

In [14]:
soup = BeautifulSoup(response.text, 'html.parser')

#### Create object for job listing cards

In [15]:
cards = soup.find_all('div', 'cardOutline')
len(cards)

15

#### Prototype the model with a single record

In [16]:
# choose first card

card = cards[0]

# use dot notation to parse specific html content

spantag = card.h2.a.span
atag = card.h2.a

# get data from the job card

job_title = spantag.get('title')
job_url = 'https://uk.indeed.com' + atag.get('href')
company = card.find('span', 'companyName').text.strip()
location = card.find('div', 'companyLocation').text.strip()
summary = card.find('div', 'job-snippet').text.strip()
summary = summary.replace('\n', ' ')
post_date = card.find('span', 'date').text
post_date = post_date[6:]
metadata = card.find('div', 'attribute_snippet').text
today = datetime.today().strftime('%Y-%m-%d')

#### define function that gets records from each card

In [64]:
def get_record(card):
    spantag = card.h2.a.span
    atag = card.h2.a
    job_title = spantag.get('title')
    job_url = 'https://uk.indeed.com' + atag.get('href')
    company = card.find('span', 'companyName').text.strip()
    location = card.find('div', 'companyLocation').text.strip()
    summary = card.find('div', 'job-snippet').text.strip()
    summary = summary.replace('\n', ' ')
    post_date = card.find('span', 'date').text.strip()
    post_date = post_date[6:]
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        metadata = card.find('div', 'attribute_snippet').text.strip()
    except AttributeError:
        metadata = ''
    
    record = (job_title, company, location, metadata, post_date, today, summary, job_url)
    
    return record

#### Loop through cards and store data in empty list called records

In [65]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

#### Getting next page

In [75]:
while True:
    try:
        url = 'https://uk.indeed.com' + soup.find('a', {'aria-label': 'Next Page'}).get('href')
    except AttributeError:
        break
        
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'cardOutline')
    
    for card in cards:
        record = get_record(card)
        records.append(record)


#### Final Product

In [79]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import random
from itertools import cycle
from urllib.parse import urlencode

def get_url(position, location):
    template = 'https://uk.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

def get_record(card):
    spantag = card.h2.a.span
    atag = card.h2.a
    job_title = spantag.get('title')
    job_url = 'https://uk.indeed.com' + atag.get('href')
    company = card.find('span', 'companyName').text.strip()
    location = card.find('div', 'companyLocation').text.strip()
    summary = card.find('div', 'job-snippet').text.strip()
    summary = summary.replace('\n', ' ')
    post_date = card.find('span', 'date').text.strip()
    post_date = post_date[6:]
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        metadata = card.find('div', 'attribute_snippet').text.strip()
    except AttributeError:
        metadata = ''
    
    record = (job_title, company, location, metadata, post_date, today, summary, job_url)
    
    return record

def main(position, location):
    records = []
    url = get_url(position, location)
    
    while True:
        
        proxy_params = {
        'api_key': '78923aa5-caac-440a-a88f-d1105626fd12',
        'url': url, 
        'render_js': True,}
        response = requests.get(
        url='https://proxy.scrapeops.io/v1/',
        params=urlencode(proxy_params),
        timeout=120,)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'cardOutline')  
    
        for card in cards:
            record = get_record(card)
            records.append(record)
        
        try:
            url = 'https://uk.indeed.com' + soup.find('a', {'aria-label': 'Next Page'}).get('href')
        except AttributeError:
            break
        
    
        for card in cards:
            record = get_record(card)
            records.append(record)
            
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title', 'Company Name', 'Location', 'Metadata', 'Post Date', 'Today', 'Job Description', 'URL'])
        writer.writerows(records)

In [80]:
main('software+consultant', 'bournemouth')